In [31]:
import numpy as np
import pandas as pd
import basedosdados as bd

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, precision_score, recall_score
from sklearn.metrics import confusion_matrix, fbeta_score, brier_score_loss, precision_recall_curve, auc

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.svm import SVC, OneClassSVM

## Dados

In [23]:
# TODO rode isto para criar o csv com vários hexágonos (O finalizado)
# main_table = bd.read_sql(query = """ SELECT * FROM `rj-cor-dev.clima_pluviometro.main_table_fields_1H_mais_frequentes`""", billing_project_id = "projeto-fgv1", use_bqstorage_api = True)
# main_table.to_csv("csvs/main_table_mult_hexag.csv")

In [24]:
main_table_k = pd.read_csv("csvs/main_table_mult_hexag.csv")

## Pré-processamento

In [32]:
main_table = main_table_k.copy()

In [33]:
main_table.fillna(0, inplace=True)

main_table["target"] = main_table["alagamento_pop"].apply(lambda x: 1 if x > 0 else 0)

main_table["id_h3"] = main_table["id_h3"].astype("category")

# Aplicar one-hot encoding na coluna "estacao_ano"
one_hot_encoder = OneHotEncoder(sparse=False)
encoded_cols = one_hot_encoder.fit_transform(main_table[["estacao_ano"]])
encoded_labels = one_hot_encoder.categories_[0]

# Adicionar as colunas codificadas ao DataFrame original
for i, label in enumerate(encoded_labels):
    main_table[f"estacao_ano_{label}"] = encoded_cols[:, i]


# Transformar a coluna id_h3 em categórica
label_encoder = LabelEncoder()
main_table["id_h3"] = label_encoder.fit_transform(main_table["id_h3"])


main_table.drop(columns=["data_hora", "estacao_ano", "alagamento_fim",
                         "estacoes", "Unnamed: 0", "alagamento_pop", "alagamento_inicio", 
                         "quinzenas", "alagamento_lat", "alagamento_long", "id_alagamento",
                         "gravidade_alagamento"], inplace=True)

In [34]:
X = main_table.drop(columns=["target"])
y = main_table["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

std = StandardScaler()

X_train_standard = std.fit_transform(X_train)
X_val_standard = std.transform(X_val)
X_test_standard = std.transform(X_test)

X_test

,id_h3,chuva_15min,chuva_1h,chuva_4h,chuva_24h,chuva_96h,estacao_ano_Inverno,estacao_ano_Outono,estacao_ano_Primavera,estacao_ano_Verão
1169904,46,0.0,0.0,0.000000,0.000000,0.200000,0.0,0.0,0.0,1.0
669389,29,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0
2774759,22,0.0,0.0,0.068917,18.331036,11.921848,0.0,0.0,0.0,1.0
4033651,1,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0
2258501,25,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
3698271,11,0.0,0.0,0.000000,0.000000,0.799812,1.0,0.0,0.0,0.0
1719309,47,0.0,0.0,0.000000,0.000000,11.417596,0.0,1.0,0.0,0.0
1514465,9,0.0,0.0,0.000000,8.804468,13.225380,0.0,0.0,0.0,1.0
3749992,26,0.0,0.0,0.000000,3.042105,39.104516,0.0,0.0,1.0,0.0


## Modelos

In [35]:
# Naive Model
# predicts majority class always

# predictid minority proportion

naive_pred = np.ones(len(y_test)) * y_train.mean()
y_pred_nv = np.where(naive_pred > 0.5, 1, 0)

# metrics
print("Naive Model")

print("R2: ", r2_score(y_test, y_pred_nv))
print("MSE: ", mean_squared_error(y_test, y_pred_nv))
print("Precision: ", precision_score(y_test, y_pred_nv))
print("Recall: ", recall_score(y_test, y_pred_nv))
print("F0.5: ", fbeta_score(y_test, y_pred_nv, beta=0.5))
recall_nv, precision_nv, th_nv = precision_recall_curve(y_test, y_pred_nv)
print("Area under Precision Recall Curve: ", auc(recall_nv, precision_nv))
print("Brier Score: ", brier_score_loss(y_test, y_pred_nv))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred_nv))

Naive Model
R2:  -0.0020379242640451434
MSE:  0.0020337795753018867
Precision:  0.0
Recall:  0.0
F0.5:  0.0
Area under Precision Recall Curve:  0.49898311021234903
Brier Score:  0.0020337795753018867
Confusion Matrix: 
 [[604046      0]
 [  1231      0]]


In [36]:
# Logistic Regression

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_standard, y_train)

y_pred_lr = logreg.predict(X_test_standard)

print("R²: ", r2_score(y_test, y_pred_lr))
print("Precison: ", precision_score(y_test, y_pred_lr))
print("Recall: ", recall_score(y_test, y_pred_lr))
print("Brier Score: ", brier_score_loss(y_test, y_pred_lr))
print("F-0.5 Score: ", fbeta_score(y_test, y_pred_lr, beta=0.5))
recall_lr, precision_lr, th_lr = precision_recall_curve(y_test, y_pred_lr)
print("Area under Precision Recall Curve: ", auc(recall_lr, precision_lr))
print("confusion_matrix: \n", confusion_matrix(y_test, y_pred_lr))

R²:  -0.037854064530184806
Precison:  0.36585365853658536
Recall:  0.048740861088545896
Brier Score:  0.0021064735649958615
F-0.5 Score:  0.1589825119236884
Area under Precision Recall Curve:  0.20623080594103244
confusion_matrix: 
 [[603942    104]
 [  1171     60]]


In [37]:
# KNN

knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_train_standard, y_train)

y_pred_knn = knn.predict(X_test_standard)

print("R²: ", r2_score(y_test, y_pred_knn))
print("Precison: ", precision_score(y_test, y_pred_knn, average="macro"))
print("Recall: ", recall_score(y_test, y_pred_knn, average="macro"))
print("Brier Score: ", brier_score_loss(y_test, y_pred_knn))
print("F-0.5 Score: ", fbeta_score(y_test, y_pred_knn, beta=0.5))
recall_knn, precision_knn, th_knn = precision_recall_curve(y_test, y_pred_knn)
print("Area under Precision Recall Curve: ", auc(recall_knn, precision_knn))
print("confusion_matrix: \n", confusion_matrix(y_test, y_pred_knn))

R²:  0.05168628613516435
Precison:  0.8202265412860429
Recall:  0.5604511991413827
Brier Score:  0.0019247385907609244
F-0.5 Score:  0.3450671607225568
Area under Precision Recall Curve:  0.3805006182441584
confusion_matrix: 
 [[603963     83]
 [  1082    149]]


In [38]:
# Neural Network

# 10 camadas ocultas com 10 neurônios cada relu
mlp = MLPClassifier(hidden_layer_sizes=(5,5), max_iter=1000, activation="relu", 
                    solver="adam", random_state=1, early_stopping=True)
mlp.fit(X_train_standard, y_train)

y_pred_nn = mlp.predict(X_test_standard)

print("R²: ", r2_score(y_test, y_pred_nn))
print("Precison: ", precision_score(y_test, y_pred_nn))
print("Recall: ", recall_score(y_test, y_pred_nn))
print("Brier Score: ", brier_score_loss(y_test, y_pred_nn))
print("F-0.5 Score: ", fbeta_score(y_test, y_pred_nn, beta=0.5))
recall_nn, precision_nn, th_nn = precision_recall_curve(y_test, y_pred_nn)
print("Area under Precision Recall Curve: ", auc(recall_nn, precision_nn))
print("confusion_matrix: \n", confusion_matrix(y_test, y_pred_nn))

R²:  -0.0012239210761784136
Precison:  0.5151515151515151
Recall:  0.01380991064175467
Brier Score:  0.0020321274391724782
F-0.5 Score:  0.0623624358033749
Area under Precision Recall Curve:  0.26344977995188396
confusion_matrix: 
 [[604030     16]
 [  1214     17]]


In [39]:
# Decision Tree

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

y_pred_dt = dt.predict(X_test)

print("R²: ", r2_score(y_test, y_pred_dt))
print("Precison: ", precision_score(y_test, y_pred_dt))
print("Recall: ", recall_score(y_test, y_pred_dt))
print("Brier Score: ", brier_score_loss(y_test, y_pred_dt))
print("F-0.5 Score: ", fbeta_score(y_test, y_pred_dt, beta=0.5))
recall_dt, precision_dt, th_dt = precision_recall_curve(y_test, y_pred_dt)
print("Area under Precision Recall Curve: ", auc(recall_dt, precision_dt))
print("confusion matrix:\n", confusion_matrix(y_test, y_pred_dt))

R²:  -0.2226327881759511
Precison:  0.3942232630757221
Recall:  0.4102355808285946
Brier Score:  0.0024815084663715953
F-0.5 Score:  0.39732494099134535
Area under Precision Recall Curve:  0.4007953677918318
confusion matrix:
 [[603270    776]
 [   726    505]]


In [40]:
# Random Forest

rf = RandomForestClassifier(n_estimators=10, criterion="entropy")
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

print("R²: ", r2_score(y_test, y_pred_rf))
print("Precison: ", precision_score(y_test, y_pred_rf))
print("Recall: ", recall_score(y_test, y_pred_rf))
print("Brier Score: ", brier_score_loss(y_test, y_pred_rf))
print("F-0.5 Score: ", fbeta_score(y_test, y_pred_rf, beta=0.5))
recall_rf, precision_rf, th_rf = precision_recall_curve(y_test, y_pred_rf)
print("Area under Precision Recall Curve: ", auc(recall_rf, precision_rf))
print("confusion matrix:\n", confusion_matrix(y_test, y_pred_rf))

R²:  0.1965788535754569
Precison:  0.7904761904761904
Recall:  0.2696994313566206
Brier Score:  0.001630658359726208
F-0.5 Score:  0.5702507729302644
Area under Precision Recall Curve:  0.5287966665312727
confusion matrix:
 [[603958     88]
 [   899    332]]


In [13]:
# SVM

svm = SVC()

svm.fit(X_train_standard, y_train)

y_pred_svm = svm.predict(X_test_standard)

print("R²: ", r2_score(y_test, y_pred_svm))
print("Precison: ", precision_score(y_test, y_pred_svm))
print("Recall: ", recall_score(y_test, y_pred_svm))
print("Brier Score: ", brier_score_loss(y_test, y_pred_svm))
print("F-0.5 Score: ", fbeta_score(y_test, y_pred_svm, beta=0.5))
recall_svm, precision_svm, th_svm = precision_recall_curve(y_test, y_pred_svm)
print("Area under Precision Recall Curve: ", auc(recall_svm, precision_svm))
print("confusion matrix:\n", confusion_matrix(y_test, y_pred_svm))


R²:  0.2526258448747565
Precison:  0.8958333333333334
Recall:  0.296551724137931
Brier Score:  0.009323806204252353
F-0.5 Score:  0.6379821958456974
Area under Precision Recall Curve:  0.5880015214160087
confusion matrix:
 [[11326     5]
 [  102    43]]


## Imbalanced Models

In [41]:
# Isolation Forest

isolation_forest = IsolationForest(n_estimators=100, contamination=0.001)
isolation_forest.fit(X_train)

y_pred_if = isolation_forest.predict(X_test)
y_pred_if = np.where(y_pred_if == -1, 1, 0)

print("R²: ", r2_score(y_test, y_pred_if))
print("Precison: ", precision_score(y_test, y_pred_if))
print("Recall: ", recall_score(y_test, y_pred_if))
print("Brier Score: ", brier_score_loss(y_test, y_pred_if))
print("F-0.5 Score: ", fbeta_score(y_test, y_pred_if, beta=0.5))
recall_if, precision_if, th_if = precision_recall_curve(y_test, y_pred_if)
print("Area under Precision Recall Curve: ", auc(recall_if, precision_if))
print("confusion matrix:\n", confusion_matrix(y_test, y_pred_if))


R²:  -0.2950790718960974
Precison:  0.2009966777408638
Recall:  0.0982940698619009
Brier Score:  0.0026285485818889533
F-0.5 Score:  0.16625446551250342
Area under Precision Recall Curve:  0.1485285297779022
confusion matrix:
 [[603565    481]
 [  1110    121]]


In [18]:
# One Class SVM

one_class_svm = OneClassSVM()
one_class_svm.fit(X_train_standard)

y_pred_ocs = one_class_svm.predict(X_test_standard)
y_pred_ocs = np.where(y_pred_ocs == -1, 1, 0)

print("R²: ", r2_score(y_test, y_pred_ocs))
print("Precison: ", precision_score(y_test, y_pred_ocs))
print("Recall: ", recall_score(y_test, y_pred_ocs))
print("Brier Score: ", brier_score_loss(y_test, y_pred_ocs))
print("F-0.5 Score: ", fbeta_score(y_test, y_pred_ocs, beta=0.5))
recall_ocs, precision_ocs, th_ocs = precision_recall_curve(y_test, y_pred_ocs)
print("Area under Precision Recall Curve: ", auc(recall_ocs, precision_ocs))
print("confusion matrix:\n", confusion_matrix(y_test, y_pred_ocs))

R²:  -41.6422356732674
Precison:  0.02059202059202059
Recall:  0.8827586206896552
Brier Score:  0.5319797838968282
F-0.5 Score:  0.025590787316566034
Area under Precision Recall Curve:  0.4397809323522356
confusion matrix:
 [[5243 6088]
 [  17  128]]


In [42]:
# Local Outlier Factor

local_outlier_factor = LocalOutlierFactor(n_neighbors=20, contamination=0.01, novelty=True)
local_outlier_factor.fit(X_train_standard)

y_pred_lof = local_outlier_factor.predict(X_test_standard)
y_pred_lof = np.where(y_pred_lof == -1, 1, 0)

print("R²: ", r2_score(y_test, y_pred_lof))
print("Precison: ", precision_score(y_test, y_pred_lof))
print("Recall: ", recall_score(y_test, y_pred_lof))
print("Brier Score: ", brier_score_loss(y_test, y_pred_lof))
print("F-0.5 Score: ", fbeta_score(y_test, y_pred_lof, beta=0.5))
recall_lof, precision_lof, th_lof = precision_recall_curve(y_test, y_pred_lof)
print("Area under Precision Recall Curve: ", auc(recall_lof, precision_lof))
print("confusion matrix:\n", confusion_matrix(y_test, y_pred_lof))